This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace. 

In [88]:
!pip install --user pandas
# !pip install -q -r ./requirements.txt
!pip install openai==0.28
!pip install langchain
!pip install langchain_core
!pip install langchain_openai
!pip install lancedb

Defaulting to user installation because normal site-packages is not writeable
  Using cached openai-0.28.0-py3-none-any.whl (76 kB)
  Attempting uninstall: openai
    Found existing installation: openai 0.28.1
    Uninstalling openai-0.28.1:
      Successfully uninstalled openai-0.28.1
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.3.12 requires langchain-core<1.0.0,>=0.3.49, but you have langchain-core 0.1.53 which is incompatible.
langchain-openai 0.3.12 requires openai<2.0.0,>=1.68.2, but you have openai 0.28.0 which is incompatible.
Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached openai-1.70.0-py3-none-any.whl (599 kB)
  Using cached langchain_core-0.3.49-py3-none-any.whl (420 kB)


  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.53
    Uninstalling langchain-core-0.1.53:
      Successfully uninstalled langchain-core-0.1.53
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.9 requires langchain-core<0.2,>=0.1.26, but you have langchain-core 0.3.49 which is incompatible.
langchain-experimental 0.0.52 requires langchain-core<0.2.0,>=0.1.24, but you have langchain-core 0.3.49 which is incompatible.
langchain-community 0.0.38 requires langchain-core<0.2.0,>=0.1.52, but you have langchain-core 0.3.49 which is incompatible.
De

In [85]:
import os
os.environ["OPENAI_API_KEY"] = "PLACE YOUR API KEY"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

from langchain.llms import OpenAI

In [119]:
import os
import openai
import json
import chromadb

import shutil
from typing import List
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from pydantic import BaseModel, Field
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma, lancedb
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores.utils import filter_complex_metadata

from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [120]:
from bs4 import BeautifulSoup

# Generating Real Estate Listings - LangChain

In [74]:
from langchain.llms import OpenAI

model_name = 'gpt-3.5-turbo'
llm = OpenAI(model_name=model_name, temperature=0)

instruction_system_prompt = """
You are an expert real estate agent in California in the USA.

    Generate at least 20 real estate listings. Utilize your imagination to generate listings.
    Ensure to include real listings as well.
    Distribute listings across the 5 California boroughs.
    The listings must be in the JSON array of dictionaries with each item in the format as shown below:

"""

sample = """
Neighborhood: Green Oaks
Price: $800,000
Bedrooms: 3
Bathrooms: 2
House Size: 2,000 sqft
Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.
Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze.
"""


In [75]:
class RealEstateListing(BaseModel):
    """
    A real estate listing.
    
    Attributes:
    - neighborhood: str
    - price: NonNegativeInt
    - bedrooms: NonNegativeInt
    - bathrooms: NonNegativeInt
    - house_size: NonNegativeInt
    - description: str
    - neighborhood_description: str
    """
    neighborhood: str = Field(description="The neighborhood where the property is located")
    price: int = Field(description="The price of the property in USD")
    bedrooms: int = Field(description="The number of bedrooms in the property")
    bathrooms: int = Field(description="The number of bathrooms in the property")
    house_size: int = Field(description="The size of the house in square feet")
    description: str = Field(description="A description of the property")
    neighborhood_description: str = Field(description="A description of the neighborhood.")  

class ListingCollection(BaseModel):
    """
    A collection of real estate listings.
    
    Attributes:
    - listings: List[RealEstateListing]
    """
    listings: List[RealEstateListing] = Field(description="A list of real estate listings")

In [76]:
# generate parsed output
parser = PydanticOutputParser(pydantic_object=ListingCollection)

In [77]:
prompt = PromptTemplate(
    template="{instruction}\n{sample}\n{format_instructions}\n",
    input_variables=["instruction", "sample"],
    partial_variables={"format_instructions": parser.get_format_instructions},
)

query = prompt.format(
    instruction=instruction_system_prompt,
    sample=sample,
)
print(query)


You are an expert real estate agent in California in the USA.

    Generate at least 20 real estate listings. Utilize your imagination to generate listings.
    Ensure to include real listings as well.
    Distribute listings across the 5 California boroughs.
    The listings must be in the JSON array of dictionaries with each item in the format as shown below:



Neighborhood: Green Oaks
Price: $800,000
Bedrooms: 3
Bathrooms: 2
House Size: 2,000 sqft
Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.
Neighbor

In [78]:
output = llm(query)

output

'{\n  "listings": [\n    {\n      "neighborhood": "Green Oaks",\n      "price": 800000,\n      "bedrooms": 3,\n      "bathrooms": 2,\n      "house_size": 2000,\n      "description": "Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.",\n      "neighborhood_description": "Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access 

In [66]:
parsed_output = json.loads(output)
parsed_output

{'listings': [{'neighborhood': 'Green Oaks',
   'price': 800000,
   'bedrooms': 3,
   'bathrooms': 2,
   'house_size': 2000,
   'description': 'Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.',
   'neighborhood_description': 'Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, co

# Storing Listings in a Vector Database

In [121]:

embedding_function = OpenAIEmbeddings()

documents = []
for index, listing in enumerate(parsed_output["listings"]):
    documents.append(
        Document(
            page_content=listing["description"],
            metadata={"id": str(index)}
        )
    )

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=100,
    length_function=len,
    add_start_index=True,
)
chunks = text_splitter.split_documents(documents)
print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

if chunks:
    sample = chunks[0]
    print(sample.page_content)
    print(sample.metadata)

if os.path.exists(CHROMA_PATH):
    shutil.rmtree(CHROMA_PATH)

embedding_function = OpenAIEmbeddings()

db = Chroma.from_documents(
    chunks, embedding_function, persist_directory=CHROMA_PATH
)
db.persist()
print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")


Split 5 documents into 11 chunks.
Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious
{'id': '0', 'start_index': 0}
Saved 11 chunks to chroma.


# Storing Listings in a Vector Database

In [122]:
questions = [
    "How big do you want your house to be?",
    "What are 3 most important things for you in choosing this property?",
    "Which amenities would you like?",
    "Which transportation options are important to you?",
    "How urban do you want your neighborhood to be?",
]

answers = [
    "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, good local schools, and convenient shopping options.",
    "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."
]



preference_text = " ".join(answers)

In [123]:
retrieved_docs = db.similarity_search(preference_text, k=3)


In [124]:
retrieved_docs

[Document(page_content='closet. Enjoy the California sunshine in the beautifully landscaped backyard with a pool and outdoor kitchen. Conveniently located near top-rated schools, shopping centers, and parks, this home offers the perfect blend of comfort and convenience.', metadata={'id': '1', 'start_index': 206}),
 Document(page_content='Luxury living awaits in this stunning 4-bedroom, 3-bathroom home in the desirable neighborhood of Sunnyvale. The spacious layout features high ceilings, a gourmet kitchen, and a master suite with a walk-in closet. Enjoy the California sunshine in the beautifully landscaped backyard with a pool and', metadata={'id': '1', 'start_index': 0}),
 Document(page_content='elevator. Relax in the expansive backyard with a pool, spa, and outdoor kitchen, perfect for entertaining guests. With stunning views of the Golden Gate Bridge and easy access to top-rated schools and upscale shopping, this home offers the epitome of upscale living.', metadata={'id': '3', 'sta

# Searching Based on Preferences

In [132]:
buyer_preferences = "Prefers natural lighting, modern kitchen, and family-friendly neighborhood."

prompt_text = "A comfortable 4-bedroom house with a spacious kitchen and a beautiful living room." 

PROMPT_TEMPLATE =\
"""
You are a real estate agent. Your task is to personalize the following property listing to better match the buyer's preferences.

Buyer Preferences:
{preferences}

Original Listing:
{listing}

Personalized Description:
"""


# Personalizing Listing Descriptions

In [128]:
def predict_response(prompt_text, PROMPT_TEMPLATE, preferences_text):
    embedding_function = OpenAIEmbeddings()
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    results = db.similarity_search_with_relevance_scores(prompt_text, k=3)
    
    if len(results) == 0 or results[0][1] < 0.7:
        print("Unable to find matching results.")
        return

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _ in results])
    
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(
        preferences=preferences_text,
        listing=context_text
    )

    print(f"\nGenerated Prompt:\n{prompt}")

    model = ChatOpenAI()
    response_text = model.predict(prompt)

    sources = [doc.metadata.get("id", None) for doc, _ in results]
    formatted_response = f"\nResponse:\n{response_text}\nSources: {sources}"
    print(formatted_response)


In [133]:
predict_response(prompt_text, PROMPT_TEMPLATE, buyer_preferences)



Generated Prompt:
Human: 
You are a real estate agent. Your task is to personalize the following property listing to better match the buyer's preferences.

Buyer Preferences:
Prefers natural lighting, modern kitchen, and family-friendly neighborhood.

Original Listing:
Luxury living awaits in this stunning 4-bedroom, 3-bathroom home in the desirable neighborhood of Sunnyvale. The spacious layout features high ceilings, a gourmet kitchen, and a master suite with a walk-in closet. Enjoy the California sunshine in the beautifully landscaped backyard with a pool and

---

Experience luxury living in this modern 5-bedroom, 4-bathroom home perched in the prestigious Hollywood Hills. The sleek design features floor-to-ceiling windows, a gourmet kitchen, and a private rooftop deck with panoramic views of the city. Entertain guests in the spacious living areas or relax in

---

Escape to paradise in this luxurious 4-bedroom, 3-bathroom home perched on the cliffs of La Jolla. The open floor pla

In [136]:
AUGMENT_PROMPT_TEMPLATE = """
You're a real estate expert helping personalize property listings to match a buyer's unique needs.

Use the following context to guide your response:

{context}

---

Now, craft a compelling and customized description that directly responds to the question below. Highlight features of the property that align with the buyer's preferences, and make your response engaging, thoughtful, and persuasive.

Question: {question}
"""


In [138]:
def predict_response(prompt_text, prompt_template_str, preferences_text):
    embedding_function = OpenAIEmbeddings()
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    results = db.similarity_search_with_relevance_scores(prompt_text, k=3)

    if len(results) == 0 or results[0][1] < 0.7:
        print("Unable to find matching results.")
        return

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _ in results])

    # Combine preferences with question if needed
    full_question = f"{prompt_text} Buyer preferences: {preferences_text}"

    # Load the prompt template
    prompt_template = ChatPromptTemplate.from_template(prompt_template_str)
    prompt = prompt_template.format(context=context_text, question=full_question)

    print("\nGenerated Prompt:")
    print(prompt)

    model = ChatOpenAI()
    response = model.predict(prompt)

    sources = [doc.metadata.get("id", None) for doc, _ in results]
    print(f"\nResponse:\n{response}\nSources: {sources}")


In [139]:
prompt_text = "Looking for a cozy home with natural light and a quiet neighborhood."
preferences = "Loves eco-friendly design, prefers open floor plan, dislikes modern minimalist interiors."
predict_response(prompt_text, AUGMENT_PROMPT_TEMPLATE, buyer_preferences)



Generated Prompt:
Human: 
You're a real estate expert helping personalize property listings to match a buyer's unique needs.

Use the following context to guide your response:

Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious

---

closet. Enjoy the California sunshine in the beautifully landscaped backyard with a pool and outdoor kitchen. Conveniently located near top-rated schools, shopping centers, and parks, this home offers the perfect blend of comfort and convenience.

---

for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.

---

Now, craft a compelling and customized description that directly responds to the question below. Highlight features of the property that align with

In [142]:
predict_response("Find me a 3-bedroom house in california under $500,000 with a pool", AUGMENT_PROMPT_TEMPLATE, buyer_preferences)



Generated Prompt:
Human: 
You're a real estate expert helping personalize property listings to match a buyer's unique needs.

Use the following context to guide your response:

closet. Enjoy the California sunshine in the beautifully landscaped backyard with a pool and outdoor kitchen. Conveniently located near top-rated schools, shopping centers, and parks, this home offers the perfect blend of comfort and convenience.

---

Luxury living awaits in this stunning 4-bedroom, 3-bathroom home in the desirable neighborhood of Sunnyvale. The spacious layout features high ceilings, a gourmet kitchen, and a master suite with a walk-in closet. Enjoy the California sunshine in the beautifully landscaped backyard with a pool and

---

Escape to paradise in this luxurious 4-bedroom, 3-bathroom home perched on the cliffs of La Jolla. The open floor plan features vaulted ceilings, a gourmet kitchen, and a master suite with ocean views. Step outside to the expansive deck overlooking the Pacific Oce